In [ ]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

( Run <code>jupyter notebook</code> under the project directory )

In [ ]:
from ppxgboost import BoosterParser as boostparser
from ppxgboost import PPBooster as ppbooster
from ppxgboost import PaillierAPI as paillier
from ppxgboost.PPBooster import MetaData
import ppxgboost.PPKey as PPKey
import random
import time
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from secrets import token_bytes
import pyope.ope as pyope

# XGBoost for Titanic Dataset

(We use this example to demenstrate how to use ppxgboost for encypting an xgboost model and query it.)

Please go to https://www.kaggle.com/c/titanic/data and download the dataset.
In the following example, the datasets are downloaded in the example directory


### Data Preparation and Train an XGBoost ML model

In [ ]:
# The pp-xgboost for titanic 
# In the following example, the datasets are downloaded in the example directory
train = pd.read_csv('../data/titanic_train.csv')
test = pd.read_csv('../data/titanic_test.csv')

# Training dataset. We skip the data exploration part ...
# Only get the features that are useful for building the ML model
X_train = train[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']]
y_train = train[['Survived']]

# Testing dataset
X_test = test[['Pclass', 'Age', 'Fare', 'SibSp', 'Parch']]

X_train.head()

In [ ]:
y_train.head()

In [ ]:
X_test.head()

In [ ]:
# Train a xgboost model 
dtrain = xgb.DMatrix(X_train, label=y_train)
params = {'eta': 0.1}
model = xgb.train(params=params, dtrain=dtrain)

# predict using the plaintext prediction
plaintext_predict = model.predict(xgb.DMatrix(X_test))

Dump the model

In [ ]:
model.dump_model('tree.txt')

### Encryption Preparation for XGBoost Model
1. Set up some metadata information for the dataset.
2. Set up the encryption materials
3. Encrypt the model
4. Encrypt the query
5. Perform the prediction
6. Decrypt the prediction

In [ ]:
# 0. Create custom data ranges
in_range = pyope.ValueRange(pyope.DEFAULT_IN_RANGE_START, 2 ** 43 - 1)
out_range = pyope.ValueRange(pyope.DEFAULT_OUT_RANGE_START, 2 ** 63 - 1)

# 1. parsing to internal tree data structure, and output feature set
min_max = boostparser.training_dataset_parser(X_test)
enc_tree, feature_set, min_max = boostparser.model_to_trees(model, min_max)

# 2. Set up encryption materials.
ppModelKey, ppQueryKey = PPKey.generatePPXGBoostKeys()

# 3. process the tree into enc_tree
ppbooster.enc_xgboost_model(ppModelKey, enc_tree, MetaData(min_max, in_range.end))


# 4. Encrypts the input vector for prediction (using prf_key_hash and ope-encrypter) based on the feature set.
ppbooster.enc_input_vector(ppQueryKey.get_prf_key(), ppQueryKey.get_ope_encryptor(), feature_set, X_test, MetaData(min_max, in_range.end))

In [ ]:
# 5. privacy-preserving evaluation.
start = time.time()
values = ppbooster.predict_binary(enc_tree, X_test)
end = time.time()
print("Elapsed Time: ", end - start)

In [ ]:
# 6. decryption
decryptions = []

for c in values:
    decryptions.append(paillier.decrypt(ppQueryKey.get_private_key(), c))

decryptions = np.array([round(x, 7) for x in decryptions])
assert len(plaintext_predict) == len(decryptions)

# if the predicted values are same (the ppxgboost might not produce same values 
#                                    as the plaintext value due to precision)
for i in range(len(plaintext_predict)):
    assert abs(plaintext_predict[i] - decryptions[i]) < 0.000001